In [3]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

# load and set API key
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

# set up Gemini
genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel('gemini-1.5-flash')

# standard prompt
def get_response(prompt):
    generation_config = genai.types.GenerationConfig(temperature=0.8)
    response = model.generate_content(prompt, generation_config=generation_config)
    return response

ModuleNotFoundError: No module named 'google'

In [1]:
import json

# Create employees
prompt1 = [
    {"parts": [{"text": """create fake employee data for a big Belgian telecom company (Proximus), and return as a json.
                It should contain 20 employees, including for each name: function, department, and e-mail.
                Make sure to have every department included. Do not return anything other than the json"""}]}
]

result = get_response(prompt1)

# Remove the markdown
clean_json = result.text[7:].strip()[:-3].strip()
# convert to dict
employee_data = json.loads(clean_json)
print(employee_data)
# get departments
departments = {employee['department'] for employee in employee_data}
print(departments)

NameError: name 'get_response' is not defined

In [48]:
prompt2 = [
    {"parts": [{"text": f"""
Search for publically available information on 20 companies who have Belgian telecom company Proximus as a client. For each of them, present a list containing:
- The company name
- The type of service they offer Proximus. when uncleat, describe their cooperation
- when this cooperation starts/started. When unavailable, return 'Date unknown'.
make sure the results are related to at least 5 departments from this list: {departments} """}]}
    ]

result2 = get_response(prompt2)
print(result2.text)

I cannot directly access and browse the internet in real-time to gather specific client information for Proximus.  This kind of data is often kept confidential due to business agreements and competitive reasons.  Publicly available information about specific client relationships is rarely this detailed.

To find this information, you would need to:

1. **Check Proximus's website:** Look for a "Clients" or "Case Studies" section.  They may highlight some larger partnerships, but rarely with the level of detail you request.

2. **Search news articles and press releases:** Use search engines (Google, Bing, etc.) and search for "Proximus" combined with keywords like "[company name] partnership," "[company name] Proximus," or "[company name] contract." This might uncover announcements of collaborations, but the specifics will likely be limited.

3. **Look at company websites:** Visit the websites of major Belgian companies.  If they have a "Partners" or "Clients" section, you might find Pro

In [166]:
from tavily import TavilyClient
import requests

company_name = "Proximus"

# Instantiating  TavilyClient
client = TavilyClient(api_key="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")

# Logo search query
response = client.search(f"{company_name} logo", include_images=True, include_image_descriptions=True)

print(response['images'])

# Choose a logo url
prompt3 = [
    {"parts": [{"text": f"""
From this list of dictionaries added after this, select the most recent company logo, according to the description. 
Only return the link. {response['images']} """}]}
    ]
result3 = get_response(prompt3)
image_url = str(result3.text).strip()
print(image_url)

# Save file extention
file_extension = os.path.splitext(image_url)[1]
print(file_extension)

# Retreive and save the image
image = requests.get(image_url)
if image.status_code == 200:
    # Open a file in write-binary mode and save the content
    with open(f"logos\{company_name}{file_extension}", "wb") as file:
        file.write(image.content)
else:
    print("Failed to retrieve the image. Status code:", image.status_code)

<string>:32: SyntaxWarning: invalid escape sequence '\{'
<>:32: SyntaxWarning: invalid escape sequence '\{'
<string>:32: SyntaxWarning: invalid escape sequence '\{'
<>:32: SyntaxWarning: invalid escape sequence '\{'
C:\Users\becod\AppData\Local\Temp\ipykernel_6968\2541001120.py:32: SyntaxWarning: invalid escape sequence '\{'
  with open(f"logos\{company_name}{file_extension}", "wb") as file:


[{'url': 'https://logos-world.net/wp-content/uploads/2023/04/Proximus-Logo.jpg', 'description': 'The design features a stylized, abstract symbol resembling interconnected loops in a rich purple color, accompanied by the text "proximus" in a bold, modern font.'}, {'url': 'https://logos-world.net/wp-content/uploads/2023/04/Proximus-Logo-History.jpg', 'description': 'The display showcases the evolution of the Proximus logo from 1994 to the present, highlighting four distinct designs used during different time periods.'}, {'url': 'https://logosdownload.com/logo/proximus-logo-1024.png', 'description': None}, {'url': 'http://img1.wikia.nocookie.net/__cb20141021140901/logopedia/images/8/8f/Proximus.png', 'description': 'The Proximus logo features the word "proximus" in a modern, stylized font, predominantly in purple, with an abstract design element that includes blue and white.'}, {'url': 'https://www.liblogo.com/img-logo/pr4107fca8-proximus-logo-file-proximus-logo-2014-svg-liblogo.png', 'de

In [152]:
from datetime import datetime

# Get the current date, time, and day of week
current_datetime = datetime.now()
day_of_week_str = current_datetime.strftime("%A")

print(f"The current date and time is {current_datetime}, today is a {day_of_week_str}.")

The current date and time is 2025-01-16 15:52:17.653709, today is a Thursday.
